# Hangman game implementation.

This module provides a Hangman game where the player guesses letters to reveal a hidden word.
The game starts with a limited number of tries, and the player must guess the word before running out of attempts.

### Approach:
- Initially, guesses are made based on the frequency of letters in the English language.
- Vowels are prioritized, with 'e' being the most common vowel.
- As more of the word is revealed, guesses are made based on recognizable patterns, sounds, and possible word completions.
- This combination of strategies aims to maximize the chances of guessing the word within the limited number of tries.

### Key Takeaways:
- Common letters are guessed first to increase the likelihood of revealing parts of the word.
- Vowels are essential in every word, making them a priority in early guesses.
- Recognizing patterns and making educated guesses as the word becomes partially revealed.

### Strategy:
- Start with most common vowel.
- Make frequency based guesses until the word is revealed enough to show patterns.
- Make educated guessses once we have enough data.

# step-1: Training the model
I wennt with Bi-LSTM as this seemed like the most appropriate model considering:
- Captures Context from Both Directions
- Better Generalization on Partial Words
- Handles Variable Length Words Efficiently
- tronger Sequential Dependencies

### We begin by preparing the data

In [30]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

with open("words_250000_train.txt", "r") as file:
    words = file.read().splitlines()

tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(words)

sequences = tokenizer.texts_to_sequences(words)

input_data = []
output_data = []

for seq in sequences:
    for i in range(1, len(seq)):
        input_data.append(seq[:i])
        output_data.append(seq[i])

max_seq_length = max(len(seq) for seq in input_data)
input_data = pad_sequences(input_data, maxlen=max_seq_length, padding='pre')

vocab_size = len(tokenizer.word_index) + 1 
output_data = to_categorical(output_data, num_classes=vocab_size)

input_data = np.array(input_data)
output_data = np.array(output_data)

print("Data prepared. Input shape:", input_data.shape, "Output shape:", output_data.shape)



Data prepared. Input shape: (1897446, 28) Output shape: (1897446, 27)


### Then we proceed to train the model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout, Bidirectional, Masking
from tensorflow.keras.callbacks import EarlyStopping

model = Sequential()

model.add(Masking(mask_value=0.0, input_shape=(max_seq_length,)))
model.add(Embedding(input_dim=vocab_size, output_dim=50, input_length=max_seq_length))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Bidirectional(LSTM(128, return_sequences=False)))
model.add(Dropout(0.3))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model.summary()
model.fit(input_data, output_data, epochs=20, batch_size=64, validation_split=0.2, callbacks=[early_stopping])


### Saving the model and tokenization for tinkering

In [ ]:
model.save("BI_LSTM.keras")
with open("tokenizer.pkl", "wb") as f:
    pickle.dump(tokenizer, f)

# Step-2: Implementing the strategy

### Breaking down the strategy in steps
- Step 2.1: First Guess is generally 'e' which would come with the frequency algorithm anyway
- Step 2.2: Determine How Much of the Word is Revealed (total length of word/number of known letters)
- Step 2.3: Narrow Down the Possible Word List
- Step 2.4: If Few Possible Words Remain, Guess the Most Frequent Letter
- Step 2.5: If > 25% of the Word is Revealed, Use LSTM Model
- Step 2.6: Fallback Strategy – Common Letter Guess
- Step 2.7: Return the Best Guess

### Implementation
- Copy the code from provided notebook and make adjustments to the guessing function as strategized above

### updated functions:

def update_possible_words(self, word_pattern):
        regex_pattern = word_pattern.replace("_", ".")
        self.current_dictionary = [word for word in self.current_dictionary if re.fullmatch(regex_pattern, word)]

    def predict_char_probabilities(self, word_pattern):
        sequence = self.tokenizer.texts_to_sequences([word_pattern])
        sequence = pad_sequences(sequence, maxlen=self.max_seq_length, padding='pre')
        lstm_probs = self.model.predict(sequence)[0]
        
        char_probabilities = {self.tokenizer.index_word.get(i, "_"): lstm_probs[i] for i in range(len(lstm_probs))}
        
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                char_probabilities[letter] = char_probabilities.get(letter, 0) + 0.1
        
        return char_probabilities
    
    def guess(self, word):
        if not self.guessed_letters:
            return 'e' 
        
        revealed = sum(1 for c in word if c != '_') / len(word)
        
        self.update_possible_words(word)
        
        if len(self.current_dictionary) < 50:
            letter_counts = collections.Counter("".join(self.current_dictionary))
            for letter, _ in letter_counts.most_common():
                if letter not in self.guessed_letters:
                    return letter

        if revealed > 0.25:
            predicted_probabilities = self.predict_char_probabilities(word)
            sorted_probabilities = sorted(predicted_probabilities.items(), key=lambda x: x[1], reverse=True)
            for letter, _ in sorted_probabilities:
                if letter not in self.guessed_letters:
                    return letter
        
        return self.fallback_guess()

    def fallback_guess(self):
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter
        return "_"
 


In [31]:
import json
import requests
import random
import string
import secrets
import time
import re
import collections
import numpy as np
import collections
import re
import time
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model
import pickle


try:
    from urllib.parse import parse_qs, urlencode, urlparse
except ImportError:
    from urlparse import parse_qs, urlparse
    from urllib import urlencode

from requests.packages.urllib3.exceptions import InsecureRequestWarning

requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

In [32]:
import pickle
import requests
import time
import collections
import re
import json
from urllib.parse import parse_qs
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

class HangmanAPI(object):

    def __init__(self, access_token=None, session=None, timeout=None, max_seq_length=10):
        with open("tokenizer.pkl", "rb") as f:
            self.tokenizer = pickle.load(f)
        
        self.model = load_model("BI_LSTM.keras")
        self.hangman_url = self.determine_hangman_url()
        self.access_token = access_token
        self.session = session or requests.Session()
        self.timeout = timeout
        self.guessed_letters = []
        
        full_dictionary_location = "words_250000_train.txt"
        self.full_dictionary = self.build_dictionary(full_dictionary_location)
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()
        
        self.current_dictionary = self.full_dictionary
        self.max_seq_length = max_seq_length

    @staticmethod
    def determine_hangman_url():
        links = ['https://trexsim.com', 'https://sg.trexsim.com']
        data = {link: 0 for link in links}
        
        for link in links:
            for _ in range(10):
                s = time.time()
                requests.get(link)
                data[link] = time.time() - s

        return sorted(data.items(), key=lambda x: x[1])[0][0] + '/trexsim/hangman'
    
    def update_possible_words(self, word_pattern):
        regex_pattern = word_pattern.replace("_", ".")
        self.current_dictionary = [word for word in self.current_dictionary if re.fullmatch(regex_pattern, word)]

    def predict_char_probabilities(self, word_pattern):
        sequence = self.tokenizer.texts_to_sequences([word_pattern])
        sequence = pad_sequences(sequence, maxlen=self.max_seq_length, padding='pre')
        lstm_probs = self.model.predict(sequence)[0]
        
        char_probabilities = {self.tokenizer.index_word.get(i, "_"): lstm_probs[i] for i in range(len(lstm_probs))}
        
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                char_probabilities[letter] = char_probabilities.get(letter, 0) + 0.1
        
        return char_probabilities
    
    def guess(self, word):
        
        
        revealed = sum(1 for c in word if c != '_') / len(word)
        
        self.update_possible_words(word)
        
        if len(self.current_dictionary) < 50:
            letter_counts = collections.Counter("".join(self.current_dictionary))
            for letter, _ in letter_counts.most_common():
                if letter not in self.guessed_letters:
                    return letter

        if revealed > 0.25:
            predicted_probabilities = self.predict_char_probabilities(word)
            sorted_probabilities = sorted(predicted_probabilities.items(), key=lambda x: x[1], reverse=True)
            for letter, _ in sorted_probabilities:
                if letter not in self.guessed_letters:
                    return letter
        
        return self.fallback_guess()

    def fallback_guess(self):
        for letter, _ in self.full_dictionary_common_letter_sorted:
            if letter not in self.guessed_letters:
                return letter
        return "_"

    def build_dictionary(self, dictionary_file_location):
        with open(dictionary_file_location, "r") as text_file:
            return text_file.read().splitlines()

    def start_game(self, practice=True, verbose=True):
        self.guessed_letters = []
        self.current_dictionary = self.full_dictionary
        
        response = self.request("/new_game", {"practice": practice})
        if response.get('status') == "approved":
            game_id, word, tries_remains = response['game_id'], response['word'], response['tries_remains']
            if verbose:
                print(f"Game started! ID: {game_id}, Tries: {tries_remains}, Word: {word}")
            while tries_remains > 0:
                guess_letter = self.guess(word)
                self.guessed_letters.append(guess_letter)
                if verbose:
                    print(f"Guessing: {guess_letter}")
                
                try:
                    res = self.request("/guess_letter", {"game_id": game_id, "letter": guess_letter})
                except Exception:
                    continue
                
                if verbose:
                    print(f"Response: {res}")
                status, tries_remains = res.get('status'), res.get('tries_remains')
                if status == "success":
                    if verbose:
                        print(f"Game {game_id} won!")
                    return True
                elif status == "failed":
                    if verbose:
                        print(f"Game {game_id} lost. Reason: {res.get('reason', 'Out of tries!')}")
                    return False
                elif status == "ongoing":
                    word = res.get('word')
        else:
            if verbose:
                print("Failed to start a new game")
        return False

    def request(self, path, args=None, method=None):
        if args is None:
            args = {}
        if self.access_token:
            args["access_token"] = self.access_token
        
        time.sleep(0.2)
        for _ in range(50):
            try:
                response = self.session.request(
                    method or "GET", self.hangman_url + path, timeout=self.timeout, params=args, verify=False
                )
                break
            except requests.exceptions.RequestException:
                time.sleep(2)
        
        result = response.json() if 'json' in response.headers['content-type'] else {}
        if result and isinstance(result, dict) and result.get("error"):
            raise HangmanAPIError(result)
        return result
    
    def my_status(self):
        return self.request("/my_status", {})

class HangmanAPIError(Exception):
    def __init__(self, result):
        self.result = result
        self.message = result.get("error_description", result.get("error", {}).get("message", result.get("error_msg", "Unknown error")))
        Exception.__init__(self, self.message)


# Step-3: Initialize

In [33]:

api = HangmanAPI(access_token="eb18bb888f06687c3fc7eb96d9d3e0", timeout=2000)


C:\Users\abagg\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\layer.py:938: UserWarning: Layer 'embedding_4' (of type Embedding) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


# Step-4: Test

In [ ]:
for _ in range(500): #adjusting the number of runs and seeing results
    api.start_game(practice=1, verbose=True)
    [total_practice_runs,total_recorded_runs,total_recorded_successes,total_practice_successes] = api.my_status() # Get my game stats: (# of tries, # of wins)
    practice_success_rate = total_practice_successes / total_practice_runs
    print('run %d practice games out of an allotted 100,000. practice success rate so far = %.3f' % (total_practice_runs, practice_success_rate))


Game started! ID: c64578ab403e, Tries: 6, Word: _ _ _ _ _ _ _ _ _ _ _ _ 
Guessing: e
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 6, 'word': '_ _ _ _ _ _ _ _ _ _ e _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Guessing: x
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ _ _ _ _ _ _ _ _ _ e _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Guessing: n
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 5, 'word': '_ n _ _ _ _ _ _ _ _ e _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Guessing: u
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 5, 'word': 'u n _ _ u _ _ _ _ _ e _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Guessing: t
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 4, 'word': 'u n _ _ u _ _ _ _ _ e _ '}
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
Guessing: s
Response: {'game_id': 'c64578ab403e', 'status': 'ongoing', 'tries_remains': 3, 'word': 'u 

# Key Takeaways:
- the strategy performance is not up to the mark
- The frequency based algorithm works well initially but is less efficient when patterns emerge as expected
- The pattern based approach (LSTM) is underperforming

- The model could be trained further to strengthen the weak point of the strategy
- Model was trained only for 20 epochs and showed no signs of plateuing
- Training the model furhter could improve the performance signifanctly

# Step-5: Submission

In [ ]:
for i in range(1000):
    print('Playing ', i, ' th game')
    #api.start_game(practice=0,verbose=False)
    # DO NOT REMOVE as otherwise the server may lock you out for too high frequency of requests
    time.sleep(0.5)